# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute ("""CREATE KEYSPACE IF NOT EXISTS project_1b WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
    
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to project_1b
try:
    session.set_keyspace('project_1b')
    
except Exception as e:
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
music_history_query = "CREATE TABLE IF NOT EXISTS music_history "
music_history_query = music_history_query + "(sessionId INT, itemInSession INT, userId INT, song TEXT, artist TEXT, length FLOAT, firstName TEXT, lastName TEXT, \
                gender VARCHAR, level TEXT, location TEXT, PRIMARY KEY (sessionId, itemInSession))"

user_history_query = "CREATE TABLE IF NOT EXISTS user_history "
user_history_query = user_history_query + "(userId INT, sessionId INT, itemInSession INT, song TEXT, artist TEXT, length FLOAT, firstName TEXT, lastName TEXT, \
                gender VARCHAR, level TEXT, location TEXT, PRIMARY KEY ((userId, sessionId), itemInSession))"

song_history_query = "CREATE TABLE IF NOT EXISTS song_history "
song_history_query = song_history_query + "(song TEXT, userId INT, firstName TEXT, lastName TEXT, artist TEXT, location TEXT, itemInSession INT, \
                gender VARCHAR, level TEXT, length FLOAT, sessionId INT, PRIMARY KEY (song, userId))"
try:
    session.execute(music_history_query)
    session.execute(user_history_query)
    session.execute(song_history_query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert music log to music history table
        music_history_query = "INSERT INTO music_history (sessionId, itemInSession, userId, song, artist, length, firstName, lastName, gender, level, location)"
        music_history_query = music_history_query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        # Insert user log to user history table
        user_history_query = "INSERT INTO user_history (userId, sessionId, itemInSession, song, artist, length, firstName, lastName, gender, level, location)"
        user_history_query = user_history_query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        # Insert song log to song history table
        song_history_query = "INSERT INTO song_history (song, userId, firstName, lastName, artist, location, itemInSession, gender, level, length, sessionId)"
        song_history_query = song_history_query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(music_history_query, (int(line[8]), int(line[3]), int(line[10]), line[9], line[0], float(line[5]), line[1], line[4], line[2], line[6], line[7]))
        session.execute(user_history_query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], float(line[5]), line[1], line[4], line[2], line[6], line[7]))
        session.execute(song_history_query, (line[9], int(line[10]), line[1], line[4], line[0], line[7], int(line[3]), line[2], line[6], float(line[5]), int(line[8])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Give list the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM music_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
    
print (t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Give list name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "SELECT artist, song, firstName, lastName FROM user_history WHERE userId = 10 AND sessionId = 182 AND itemInSession <= 100"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'User First Name', 'User Last Name'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])

print (t)                    

+-------------------+------------------------------------------------------+-----------------+----------------+
|       Artist      |                         Song                         | User First Name | User Last Name |
+-------------------+------------------------------------------------------+-----------------+----------------+
|  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie     |      Cruz      |
|    Three Drives   |                     Greece 2000                      |      Sylvie     |      Cruz      |
| Sebastien Tellier |                      Kilometer                       |      Sylvie     |      Cruz      |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      Sylvie     |      Cruz      |
+-------------------+------------------------------------------------------+-----------------+----------------+


In [12]:
## Give list user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT firstName, lastName FROM song_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['User First Name', 'User Last Name'])
for row in rows:
    t.add_row([row.firstname, row.lastname])

print (t)

+-----------------+----------------+
| User First Name | User Last Name |
+-----------------+----------------+
|    Jacqueline   |     Lynch      |
|      Tegan      |     Levine     |
|       Sara      |    Johnson     |
+-----------------+----------------+


### Drop the tables before closing out the sessions

In [12]:
## Drop the tables before closing out the sessions
drop_music_query = "DROP TABLE music_history"
drop_user_query = "DROP TABLE user_history"
drop_song_query = "DROP TABLE song_history"
try:
    rows = session.execute(drop_music_query)
    rows_1 = session.execute (drop_user_query)
    rows_2 = session.execute (drop_song_query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [13]:
session.shutdown()
cluster.shutdown()